In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE131835"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE131835"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE131835.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE131835.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE131835.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from adipose tissues
# The description mentions using Affymetrix Clariom S Microarray to analyze gene expression
# It's not just miRNA or methylation data, so gene expression is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Height data is available in the row 5 as 'height(cm): XXX'
trait_row = 5  # height is available in row 5

# Age data is available in row 3 as 'age: XX'
age_row = 3   # age is available in row 3

# Gender/Sex data is available in row 2 as 'Sex: Male/Female'  
gender_row = 2  # gender is available in row 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert height values to numeric (continuous) format."""
    try:
        # Extract the number after the 'height(cm):' prefix
        if ":" in value:
            height_str = value.split(":", 1)[1].strip()
            return float(height_str)
        else:
            return None
    except (ValueError, IndexError):
        return None

def convert_age(value):
    """Convert age values to numeric (continuous) format."""
    try:
        # Extract the number after the 'age:' prefix
        if ":" in value:
            age_str = value.split(":", 1)[1].strip()
            return float(age_str)
        else:
            return None
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """Convert gender to binary format (0 for female, 1 for male)."""
    try:
        if ":" in value:
            gender_str = value.split(":", 1)[1].strip().lower()
            if "female" in gender_str:
                return 0
            elif "male" in gender_str:
                return 1
            else:
                return None
        else:
            return None
    except (ValueError, IndexError):
        return None

# 3. Save Metadata - Initial Filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None, which it is in this case
if trait_row is not None:
    # Create a sample characteristics dataframe in a format that works with geo_select_clinical_features
    # Create a dictionary with the sample characteristics data
    sample_chars = {0: ['tissue: Visceral', 'tissue: SubCut'], 
                    1: ['group: CWS', 'group: CWL', 'group: CONTROL', 'group: CONTROl'], 
                    2: ['Sex: Male', 'Sex: Female'], 
                    3: ['age: 51', 'age: 64', 'age: 62', 'age: 78', 'age: 47', 'age: 59', 'age: 57', 'age: 58', 
                        'age: 53', 'age: 49', 'age: 54', 'age: 60', 'age: 56', 'age: 41', 'age: 76', 'age: 81', 
                        'age: 48', 'age: 65', 'age: 68', 'age: 72'], 
                    4: ['tumour: Oesophageal adenocarcinoma', 'tumour: Oesophageal SCC', 'tumour: Gastric adenocarcinoma', 
                        'tumour: N/A', 'tumour: Gastric'], 
                    5: ['height(cm): 178', 'height(cm): 170', 'height(cm): 166', 'height(cm): 160', 'height(cm): 180', 
                        'height(cm): 163', 'height(cm): 183', 'height(cm): 172', 'height(cm): 169', 'height(cm): 158', 
                        'height(cm): 173', 'height(cm): 193', 'height(cm): 152', 'height(cm): 167', 'height(cm): 168', 
                        'height(cm): 177', 'height(cm): 165', 'height(cm): 179', 'height(cm): 190']}
    
    # Create sample IDs (columns) equal to the maximum number of samples
    max_samples = max(len(values) for values in sample_chars.values())
    sample_ids = [f"Sample_{i+1}" for i in range(max_samples)]
    
    # Create a DataFrame with NaN values
    clinical_data = pd.DataFrame(index=range(max(sample_chars.keys()) + 1), columns=sample_ids)
    
    # Fill the DataFrame with sample characteristics data
    for row_idx, row_values in sample_chars.items():
        for col_idx, value in enumerate(row_values):
            if col_idx < len(sample_ids):
                clinical_data.iloc[row_idx, col_idx] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(pd.DataFrame(preview))
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
# Analyzing dataset files and structure
import os

# List files in the cohort directory to understand what's available
print("Files in cohort directory:")
try:
    cohort_files = os.listdir(in_cohort_dir)
    for file in cohort_files:
        print(f"- {file}")
except Exception as e:
    print(f"Error accessing directory: {e}")

# Check for common GEO file patterns
soft_file = None
matrix_file = None
family_file = None
for file in os.listdir(in_cohort_dir) if os.path.exists(in_cohort_dir) else []:
    if file.endswith(".soft") or file.endswith(".soft.gz"):
        soft_file = os.path.join(in_cohort_dir, file)
    elif file.endswith("_family.soft.gz") or file.endswith("_family.soft"):
        family_file = os.path.join(in_cohort_dir, file)
    elif file.endswith("_matrix.txt") or file.endswith("_matrix.txt.gz"):
        matrix_file = os.path.join(in_cohort_dir, file)

print(f"SOFT file: {soft_file}")
print(f"Family file: {family_file}")
print(f"Matrix file: {matrix_file}")

# Load the sample characteristics from the appropriate file
if soft_file and os.path.exists(soft_file):
    # Parse SOFT file to get sample characteristics
    with open(soft_file, 'r') as f:
        lines = f.readlines()
    
    # Extract sample characteristics
    sample_chars = []
    current_section = None
    for line in lines:
        if line.startswith("!Sample_"):
            key = line.split("=")[0].strip().replace("!Sample_", "")
            value = line.split("=")[1].strip() if "=" in line else ""
            if key == "table_begin":
                current_section = "sample_table"
                sample_chars = []
            elif key == "table_end":
                current_section = None
            elif current_section == "sample_table":
                sample_chars.append(line.strip())
            
    # Create DataFrame from sample characteristics
    if sample_chars:
        import io
        sample_table = io.StringIO("\n".join(sample_chars))
        clinical_data = pd.read_csv(sample_table, sep="\t")
    else:
        clinical_data = pd.DataFrame()
else:
    # Try to find sample characteristics in other files
    sample_chars_file = None
    for file in os.listdir(in_cohort_dir) if os.path.exists(in_cohort_dir) else []:
        if "sample" in file.lower() and "char" in file.lower():
            sample_chars_file = os.path.join(in_cohort_dir, file)
            break
    
    if sample_chars_file and os.path.exists(sample_chars_file):
        clinical_data = pd.read_csv(sample_chars_file)
    else:
        # Last resort - look for any CSV file that might contain clinical data
        csv_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.csv')] if os.path.exists(in_cohort_dir) else []
        if csv_files:
            clinical_data = pd.read_csv(os.path.join(in_cohort_dir, csv_files[0]))
        else:
            clinical_data = pd.DataFrame()

# Display sample characteristics to make informed decisions
print("\nSample characteristics data shape:", clinical_data.shape)
print("Sample characteristics preview:")
print(clinical_data.head())

# Get unique values for each row to identify relevant variables
unique_values = {}
for i in range(len(clinical_data)):
    if i < clinical_data.shape[0]:
        values = set(clinical_data.iloc[i, 1:].dropna().unique())
        unique_values[i] = values
        print(f"Row {i}: {values}")

# 1. Gene Expression Data Availability
# Assuming gene expression data is available (can be overridden if evidence suggests otherwise)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify rows for trait, age, and gender
trait_row = None
age_row = None
gender_row = None

# Check each row to find trait, age, and gender information
for i in unique_values:
    values_str = ' '.join([str(v) for v in unique_values[i]])
    row_data_str = ' '.join([str(x) for x in clinical_data.iloc[i, :].values if pd.notna(x)])
    
    # Looking for height information
    if any(h in row_data_str.lower() for h in ['height', 'cm', 'meter', 'tall', 'stature']):
        trait_row = i
        print(f"Found Height information in row {i}")
    
    # Looking for age information
    if any(a in row_data_str.lower() for a in ['age', 'year', 'yrs', 'yo']):
        age_row = i
        print(f"Found Age information in row {i}")
    
    # Looking for gender information
    if any(g in row_data_str.lower() for g in ['gender', 'sex', 'male', 'female']):
        gender_row = i
        print(f"Found Gender information in row {i}")

# 2.2 Define conversion functions for each variable
def extract_value(s):
    """Extract value after colon if present."""
    if isinstance(s, str) and ':' in s:
        return s.split(':', 1)[1].strip()
    return s

def convert_trait(value):
    """Convert height value to float (continuous)."""
    if pd.isna(value):
        return None
    
    value = extract_value(value)
    if isinstance(value, str):
        # Extract numeric part from the string
        try:
            import re
            nums = re.findall(r'\d+\.?\d*', value)
            if nums:
                return float(nums[0])
        except:
            pass
    
    try:
        return float(value)
    except:
        return None

def convert_age(value):
    """Convert age value to float (continuous)."""
    if pd.isna(value):
        return None
    
    value = extract_value(value)
    if isinstance(value, str):
        # Extract numeric part from the string
        try:
            import re
            nums = re.findall(r'\d+\.?\d*', value)
            if nums:
                return float(nums[0])
        except:
            pass
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    value = extract_value(value)
    if isinstance(value, str):
        value = value.lower()
        if any(f in value for f in ['female', 'f', 'woman', 'girl']):
            return 0
        elif any(m in value for m in ['male', 'm', 'man', 'boy']):
            return 1
    return None

# 3. Save metadata about data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract and save clinical features if trait data is available
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers in the dataset
# The identifiers follow the format "ENSG00000000003_at", "ENSG00000000005_at", etc.
# ENSG identifiers are Ensembl gene IDs, not standard human gene symbols
# They need to be mapped to official gene symbols for better interpretability

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. After analyzing the gene identifiers and gene annotation data:
# - In gene_data, the IDs are like "ENSG00000000003_at" 
# - In gene_annotation, the matching column is "ID" which has the same format
# - The gene symbols are stored in the "ORF" column

# 2. Create gene mapping dataframe with the two relevant columns
gene_mapping = gene_annotation[['ID', 'ORF']]
gene_mapping = gene_mapping.rename(columns={'ORF': 'Gene'})
gene_mapping = gene_mapping.dropna()

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relationship as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene expression data after mapping
print("\nGene expression data preview after mapping:")
print(gene_data.shape)
print(gene_data.index[:10])  # Show first 10 gene symbols